## Representation d'objects spatiaux sous-forme de graphes d'évolution


In [2]:
# ##################
# # Methodology : 
# """ Fabio Guttler, Dino Ienco, Jordi Nin, Maguelonne Teisseire, and Pascal Poncelet.
# graph-based approach to detect spatiotemporal dynamics in satellite image time series.
# ISPRS Journal of Photogrammetry and Remote Sensing, 130 :92–107, 2017.
# """
###################

In [3]:
# import Pkgs
# -*- coding: utf-8 -*-
from __future__ import division
import os
import copy
import glob
from osgeo import gdal, gdalconst
from osgeo.gdalconst import * 
import numpy as np
import operator
#import plotly.plotly as py
#import plotly.graph_objs as go
import seaborn as sns;sns.set()
import networkx as nx
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt1
from graphviz import Digraph
#import matplotlib
#matplotlib.use('Agg')
from matplotlib import colors
import time
import csv
import json  
from sklearn.metrics.cluster import normalized_mutual_info_score  as nmi
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from pandas import Series, DataFrame

### STEP I : STIS images Segementation

In [2]:
#fonction de recuperation de tous les segments de lensemble des images
def allSegment(filename,data_container):
	current_image = gdal.Open(filename, GA_ReadOnly)	
	# recuperation de la 1re bande
	band = current_image.GetRasterBand(1)	
	# nombre de lignes et colonnes
	nb_cols= current_image.RasterXSize
	nb_rows = current_image.RasterYSize
	Imsize=nb_cols*nb_rows
	#filename_array=current_image.ReadAsArray()
	#affichage du nombre de ligne et de colonne
	print (nb_cols, nb_rows)
	#affichage de la taille de l'image
	current_image_size=  nb_cols*nb_rows
	# recuperation du nom de fichier
	chemin, filenam = os.path.split(filename)	
	# recuperation du nom du fichier sans extension
	filenam = filenam.split('.tif')[0]	
	#recuperation de la partie du nom du fichier representant sa periode d'acquisition
	#periode=filenam.split("-")[0]	
	periode=filenam.split("_")[0]
	print ("periode", periode)
	#structure de donnees dans laquelle sera stockee et les dates d'acquisiton, et les ids des segments et les ids des pixels qui s'y trouvent	
	# recuperation des ids de chaque pixel de limage
	id=0 	
	matrix = band.ReadAsArray()
	#print len(matrix1)
	# recuperation de la valeur du pixel en fonction des coordonnees de l'image et du segment auquel il appartient	
	for xcoord in range(matrix.shape[0]):
		for ycoord in range(matrix.shape[1]):
			#recuperation de l'id de chaque pixel de l'image
			obj_id = matrix[xcoord][ycoord]
			#matrix1.append(obj_id)
			# chaque  objet est identifie par un id + la periode d'acquisition de l'image à la quelle il appartient
			if (obj_id, periode) not in data_container:
				# structur de donnees contenant l'id (tuple) de chaque segement et l'id des pixels qui le composent
				data_container[(obj_id, periode)] = []
			data_container[(obj_id,periode)].append(id)
			id+=1
	return nb_cols,nb_rows


In [3]:
# img = '/home/rodrique/Bureau/Mise_En_Relation/data/SENTINEL2A_20160804_100613-199_L2A_T32SPF_D_V1-3_FRE_MULTI_.tif'
# data_container = {}
# allSegment(img,data_container)

### STEP II : Oject Selection

In [4]:
##############################################################################################################################
# Pour chaque pixel,nous retenons uniquement le plus grand objet(objet candidat) qui le couvre.
# Cela permet d'eviter la selection des objets de tailles tres reduites
def CandidateObject(data_container, candidate_object): 	
	#object_container contient chqaue pixel avec avec les segments dont il est element
	object_container={}
	# recuperation de tous les pixels de l'ensemble des objets obtenus dans allSegment()
	for obj in data_container:
		for pix in data_container[obj]:
			#allpixels.append(pix)
			# verification de la presence du pixel dans object_container pour eviter les redondances
			if not (pix  in object_container):
				object_container[pix] =[]
			#object_container[(pix)].append([(obj),data_container1[obj]])	
			# ajout de l'objet + sa taille
			object_container[pix].append((obj,len(data_container[obj])))	
	# pour chaque pixel, un seul objet est retenu: l'objet ayant la plus grande taille	
	for pix in object_container:
		obj_size=object_container[pix]
		obj_size = sorted(obj_size, key=operator.itemgetter(1), reverse=True)
		candidate_object[obj_size[0][0]]=data_container[obj_size[0][0]]	
		#if candidate_object[obj_size[0][1]]==1:
			#del candidate_object[obj_size[0][0]]

In [5]:
# candidate_object={}
# CandidateObject(data_container,candidate_object )
# candidate_object

In [6]:
#fonction permettant de calculler l'intersection entre deux ensemble de pixels		
def intersection(a,b):
	return list(set(a) & set(b))

# fonction permettant de calculler la difference entre deux ensemble de pixels
def set_difference(a,b):
	return list(set(a) - set(b))


In [7]:
def computeScore(obj, PAC):
	#print obj
	inters = intersection(obj, PAC)
	if len(inters) == 0:
		return len(obj)
	else:
		diff = set_difference(obj, PAC)
		return float(len(diff)) / len(obj)

In [8]:
# fonction permettant la selection des objets de references.
# elle prend en paramettre l'ensemble des objets candidats,et la valeur de alpha et retoune le PAC 
def SelectionObjet(hash_obj, alpha,PAC):
	result_segment = [] #liste contenant les objets de references selectionnes
	#PAC = [] #liste contenant les pixels de chaque objet de reference selectiones	
	obj_size = [] #liste triee par ordre decroissant contenant chaque objet et son poids(nombre de pixels)
	
	for k in hash_obj:
		tup = [str(a) for a in k]
		#obj_size contient chaque objet, le couple (objet, taille(objet))
		obj_size.append([k, len(hash_obj[k])]) 			
	obj_size = sorted(obj_size, key=operator.itemgetter(1), reverse=True)
	
	
	g_id=1 #represente le numero de l'objet de reference suivant l'ordre de selection
	# condition sur le processus de selection des objets
	to_continue = True #
	#tant que la condition est verifiee, le processus continu
	while (to_continue):
		# liste temporaire contenant l'ensemble des objets avec leur score obtenu avec le PAC
		temp_list = []		
		# pour chaque objet candidat, on calcul son score par rapport a la partie deja couverte(pac)
		for i in range(len(obj_size)):
			score = computeScore(hash_obj[obj_size[i][0]], PAC)
			temp_list.append([obj_size[i][0], score])
		temp_list = sorted(temp_list, key=operator.itemgetter(1), reverse=True)	
		# on commence par retenir les objets de grandes tailles,
		#et par suite uniquement ceux dont le score avec le pac est sup a alpha
		temp_list = [el for el in temp_list if el[1]>alpha]
		#temp_list = filter(temp_list, lambda x: x[1] != 0)
		# verification de la condition d'arret lorsque la liste devient vide		
		if len(temp_list) == 0:
			to_continue = False
		else:
			
			# el2add represente l'objet a ajouter a la fin de chaque boucle
			el2add = temp_list[0][0]
			# l'objet est ensuite ajoute dans la liste des objets de refs
			result_segment.append([(el2add,g_id)])
			g_id+=1
            # ajout des valeurs(pixels) de l'objet dans pac
			PAC.extend(hash_obj[el2add])
			#print "np.uniq", len(np.unique(PAC))
			pacCov= np.unique(PAC)
			#print ("len(pacCov)", len(pacCov))
			#print "pacCov", len(pacCov)/199698 * 100,"%"
		#print "min temp_list %d" % temp_list[-1][1]
		#id+=1
		#print ("temp_list size %d" % len(temp_list))
		#print ("PAC size %d" % len(PAC))
		#print ("size res %d" % len(result_segment))
		
		#print ("========")
	#del result_segment[0]		
	#print result_segment
	return 	result_segment

In [9]:
# PAC = []
# alpha = 0.6
# SelectionObjet(candidate_object, alpha,PAC)

### STEP III : Graphe Node Selection

In [11]:
#fonction de selection des noeuds en chevauchement pour la construction du graphe
# elle prends en paramettre les objets de references, les objets candidats, 
# et deux criteres de selection sigma1 et sigma2 
def selectNode(result_segment,hash_obj,sigma1,sigma2):
	# listeGraph contient chaque periode et l'ensemble de ces objets de references 
	listeGraph=[]
	wholeGraphCov=[]
	#chaque objet de reference est identifie par sa periode d'acquisition
	for refObj in result_segment:	
		# structure regoupant les objet en fonction de leur periode d'acquisition de chaque obref
		#print "refObj[0][1]", refObj[0][1]
		listeNode={}
		listeNode[refObj[0][0][1]]={}
		listeNode[refObj[0][0][1]][refObj[0][0]]=hash_obj[refObj[0][0]]	
		#print "listeNode*******", listeNode
		for obj in hash_obj:
			#print "hash_obj[obj]", hash_obj[obj]
			#l'intersection n'est faite qu'entre deux objets dont les periode d'acquisition sont diff
			if obj[1]!= refObj[0][0][1]:		
				#calcul d'intersection entre les objets de ref et les objs candidats					
				inter=set(hash_obj[refObj[0][0]]).intersection(set(hash_obj[obj]))
				#print len(inter)/len(hash_obj[refObj]),  len(inter)/len(hash_obj[obj])
				# verification de la condition de selection d'un objet candidat comme noeud du graphe
				if len(inter)/len(hash_obj[obj])>=sigma1 or len(inter)/len(hash_obj[refObj[0][0]])>=sigma2:
				#print "ok"
				#if len(inter)> 0:
					#une periode donnee + l'ensemble de ces segements ne sont ajoutees qu'une et une seule fois
					if not (obj[1] in listeNode) :
						listeNode[obj[1]]={}
					listeNode[obj[1]][obj]=hash_obj[obj]
					wholeGraphCov.extend(hash_obj[obj])
					
					#print "listeNode", listeNode
		# construction de la liste des graphes
		len_wholeGraphCov=np.unique(wholeGraphCov)
		#print "len_wholeGraphCov", len(len_wholeGraphCov)
		listeGraph.append((refObj,listeNode))		
		#print "*****", 	listeGraph
	return listeGraph	

In [12]:
##############################################################################################################################
# generation de la matrice: la valeur radiométrique initiale de chaq pixel est remplacee par la valeur max ou
# le nombre d'apparition dans le graphe qui le couvre le plus d fois (seuil S)
# matrix2 : liste contenant les tuples (pixel, seuil)
# grid : represente une matrice vide a la taille de l'image  
def heatmap(matrix2,grid):	
	grid1=grid
	grid2 = np.copy(grid)
	i=0
	for xcoord in range(grid.shape[0]):
		for ycoord in range(grid.shape[1]):		
			# ind est un tuple (pixel, S)
			for ind in matrix2:
				# verification de l'equivalence des pixels
				if ind[0]==i:
					#si nous avons les memes pixels, alors sa valeur radiometrique est remplacer par la valeur  de seuil S
					grid[xcoord][ycoord]=int(ind[1])
					if int(ind[1])>0:	
						# grid2 est une matrice binaire, 1 si le pixel apparait au moins ue fois dans un graph, et 0 sinon
						grid2[xcoord][ycoord]=1			
			i+=1			
	return grid, grid2



### STEP IV : Build Grpahes

In [14]:
###################################################################################
# fonction permettant d'affecter a chaq pixel, la valeur du graph qui le contient
# graphId, ensemble de tuple compose de (pixel, id_graph)
# grid represente la nouvelle matrice contenant les pixels avec leur nouvelles valeurs
def shape_array(graphId,grid):	
	i=0
	for xcoord in range(grid.shape[0]):
		for ycoord in range(grid.shape[1]):			
			for ind in graphId:
				if ind[0]==i:					
					grid[xcoord][ycoord]=int(ind[1])				
			i+=1
			
	return grid	

In [15]:
##############################################################################################################################		
# fonction permettant le passage d'une matrice binaire (0,1) a une matrice (0, refObj_id)
# zeroOneArray==> matrice binaire (au cas ou l'on desire mettre à 0 certaine valeur seuil)
# bbxArray ==> matrice contenant les valeurs des oobjets de references
def zero_one_to_bbx(zeroOneArray, bbxArray):	
	zeroOneArray=np.array(zeroOneArray)
	bbxArray=np.array(bbxArray)
	zeroOneToBbx=zeroOneArray*bbxArray
	
	return zeroOneToBbx

In [16]:
#####################################################################################################
# fonction permettant de generer un fichier tiff a partir d'une matrice
# inputfile represente le fichier de georeferencement
# array matrice a ecrire dans le tiffile
# outFileName designation du fichier de sortie
def array_to_raster(inputfile,array,outFileName):
	ds = gdal.Open(inputfile,GA_ReadOnly)
	band = ds.GetRasterBand(1)
	#print band
	arr = band.ReadAsArray()
	[cols, rows] = arr.shape	
	driver = gdal.GetDriverByName("GTiff")
	#outFileName="/media/rodrique/DATA/SENTINEL-2-10m_multi_band_studyarea/out.tif"	
	outdata = driver.Create(outFileName, rows, cols, 1, gdal.GDT_Float32)
	#outdata = driver.Create(outFileName, rows, cols, 1, gdal.GDT_UInt16)
	outdata.SetGeoTransform(ds.GetGeoTransform())##sets same geotransform as input
	outdata.SetProjection(ds.GetProjection())##sets same projection as input
	outdata.GetRasterBand(1).WriteArray(array)
	#outdata.GetRasterBand(1).SetNoDataValue(10000)##if you want these values transparent
	outdata.FlushCache() ##saves to disk!!
	outdata = None
	band=None
	ds=None



In [17]:
# function de construction des arretes, 
# elle prend en paramettre l'ensemble des noeuds generes dans selectNode
def getEdges(listeGraph):
	# liste contenant l'ensemble des graphes, c-a-d, chaque objRef avec l'ensemble des noeuds dont il est en chevauchement
	listGraphEdge=[]		
	for graph in listeGraph:
		#print ('==========')
		#print "graphhh", graph[0]
		#
		listeTime=graph[1].keys()
		#print "listeTime", listeTime 
		#ordonner les cles par ordre croissant (suivant les dates d'acquisitions)
		listeTime=sorted(listeTime, reverse=False)
		#print "shortedlisteTime", listeTime 
		# listeEdge contient la liste des arretes
		listeEdge=[] 
		for i in range (len(listeTime)-1):	
			# image1 correspond a l'image acquise au temps t
			image1=listeTime[i]
			#print "image1", image1
			#print "image1", len(graph[1][image1])
			# image2 correspond a l'image acquise au temps t + 1
			image2=listeTime[i+1]
			#print "image2", image2
			#print "image2", len(graph[1][image2])
			for obj1 in graph[1][image1]:
				#print "graph[1][image1]", graph[1][image1] 
				for obj2 in graph[1][image2]:
					#calcul de l'intersection entre chaque objet de l'image 1 et ceux de l'image 2.
					inter=set(graph[1][image1][obj1]).intersection(set(graph[1][image2][obj2]))
					#print set(obj1), set(obj2)
					#print "+++++", len(inter)
					if len(inter)>0:
						#relier l'obj1 et obj2 par une arrete ssi ils partagent des pixels en communs
						edge=(obj1,obj2)
						listeEdge.append(edge)			
		# listGraphEdge contient chaque objet de reference et les noeuds qui lui  sont associés
		listGraphEdge.append((graph[0][0],listeEdge))
	
	return listGraphEdge

In [18]:
#function pour la construction des graphes, elle prend en parametre, la liste contenant les graphes
# et le chemin du repertoire dans lequel on souhaite enregistrer les graphes
def display_graph(all_graph,outpath):	
	for my_graph in all_graph: 
		# Digraph est un module graphviz, permettant de construire les graphes
		dot = Digraph(comment ='MyGraph')
		#print my_graph[0]
		graph_id=repr(my_graph[0][0])+'_'+repr(my_graph[0][1])
			#print "graph_id", my_graph[0]
		for edges in my_graph[1]:	
			#stredg0=repr(edges[0])
			nod0=repr(edges[0])
			nod1=repr(edges[1])			
			if (nod0==repr(my_graph[0][0])):
				dot.node(nod0,color='lightblue2', style='filled')
			else:	
				dot.node(nod0)
			dot.edge(nod0,nod1)
			#dot.render(filename=pathDrawGraph+graph_id)	
		dot.render(outpath+graph_id+'')
		

#### Parameters

In [19]:
# Initialize parameters
alpha = 0.5
sigma1 = 0.8
sigma2 = 0.6
allSegmentObject={}
candidate_object={}
allpixels=[]
PAC=[]
all_graph_pix=[]
my_graph_data={}
pix_object={}
allGraphPix=[]
graphOverlap=[]
allGraph={}
seuilDatas={}
graphDatas={}
graphDic={}
matrix=[]
resultSegmentLength=[]
myData = []
#matpac=[]
mat1=[]
mat2=[]
mat3=[]
mat4=[]
graphValue=[]
Occupation={}

def main(directory,inputfile,output):
    
    for img in directory:
        nb_cols,nb_rows = allSegment(img,allSegmentObject)
    CandidateObject(allSegmentObject, candidate_object)
    selected_objects = SelectionObjet(candidate_object,alpha,PAC)
    ListGraph= selectNode(selected_objects,allSegmentObject,sigma1,sigma2)
    
    edges = getEdges(ListGraph)
    display_graph(edges, output)
        
    grid=np.zeros(shape=(nb_rows,nb_cols))
    multiValue,zeroOne=heatmap(matrix,grid)

    # chaque pixel contient la valeur du graphe qui le couvre
    grid_=np.zeros(shape=(nb_rows,nb_cols))
    shape=shape_array(graphValue,grid_)
    zeroOneToBbx=zero_one_to_bbx(zeroOne, shape)

    # generation fichiers tiff 
    array_to_raster(inputfile,zeroOneToBbx,output+'Bbx.tif')


In [21]:
inputfile = "20180128_multi.tif" # input image : for img dimension/shape
output = "graph_path"  # path to store graphs
directory = glob.glob(".../segmented/*_merged.tif") # segmented images
main(directory,inputfile,output)

52 53
periode 20181030
52 53
periode 20181229
52 53
periode 20180227
52 53
periode 20181124
52 53
periode 20180826
52 53
periode 20180930
52 53
periode 20180128
